In [92]:
from keras import Model
from keras.models import Sequential, Layer
from keras.layers import Dense, MaxPool2D, Dropout,MaxPool2D, Activation,BatchNormalization, InputLayer, Embedding, Flatten, Conv2D,concatenate, MaxPooling2D, Input, TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
import keras.utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.layers import Flatten, LSTM
import numpy as np
from keras import objectives
from __future__ import absolute_import

In [117]:
s1 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s2 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s3 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s4 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s5 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
s6 = np.random.rand(10,64,64,3).reshape(-1,64,64,3)
data = np.array([s1,s2,s3,s4,s5,s6]).reshape(-1,10,64,64,3)
label = np.array([0,1,2,3,4,5]).reshape(-1)
print(s1.shape)
print(data.shape)
print(label.shape)
label = keras.utils.to_categorical(label, num_classes=None)
K.set_image_data_format('channels_last')
K.image_data_format()

(10, 64, 64, 3)
(6, 10, 64, 64, 3)
(6,)


'channels_last'

In [409]:
xx = np.random.rand(10,512).reshape(-1,10,512)

In [413]:
inp = Input(shape = (10,512))
h_layer = LSTM(512, return_sequences=True, dropout=0.2, name='lstm_1')(inp)
mod = Model(x, h_layer)
mod.compile(loss= loss_func, optimizer = 'adam', metrics=['accuracy'])#E2 Objective Covered
mod.summary()

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_16:0", shape=(?, 10, 512), dtype=float32) at layer "input_16". The following previous layers were accessed without issue: []

In [414]:
class CustomRegularization(Layer):
    def __init__(self, **kwargs):
        super(CustomRegularization, self).__init__(**kwargs)

    def call(self ,x ,mask=None):
        inp = InputLayer(input_shape = (10,512))
        h_layer = LSTM(512, return_sequences=True, dropout=0.2, name='lstm_1')(inp)
        mod = Model( x, h_layer)
        mod.compile(loss= loss_func, optimizer = 'adam', metrics=['accuracy'])#E2 Objective Covered
        return h_layer

    def compute_output_shape(self, input_shape):
        print(input_shape)
        return ((None, 10, 512))

In [415]:
def min_max_norm(mat):
    min_m = min(mat)
    max_m = max(mat)
    normalized =[((x-min_m)/(max_m-min_m)) for x in mat];
    return normalized

In [416]:
def cossy(x_mat):
#     x_mat = keras.backend.get_value(x_mat)
#     k_var = K.placeholder(shape=(10,1), dtype='float32')
#     print((x_mat).eval(session=k.get_session()))
#     apex_vec = x_mat[-1][:]
#     print(type(apex_vec))
#     dotting = x_mat.dot(apex_vec)
#     dotting = np.array([col.dot(apex_vec) for col in x_mat])
#     print(keras.backend.int_shape(x_mat))
#     dotting = np.array([K.dot(x_mat[i][:], apex_vec) for i in range(10)])
#     print(dotting)
#     matrix_norms = np.linalg.norm(x_mat, axis=1)
#     vector_norm = np.linalg.norm(apex_vec)
#     matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
#     coss_mat = np.divide(dotting, matrix_vector_norms)
#     print("coss_mat: ",coss_mat.shape)
#     nrm_arr = np.array(min_max_norm(coss_mat))
#     print("nrm_arr: ",nrm_arr.shape)
#     return nrm_arr

SyntaxError: unexpected EOF while parsing (<ipython-input-416-496f46d5b5e7>, line 19)

In [417]:
def cossimilasrity(x):
#     tt = K.variable(np.random.rand(10,512).astype('f').reshape(10,512))
#     tt = np.random.rand(10,512).astype('f').reshape(10,512)
#     print(tt.shape)
#     y = keras.layers.Lambda(cossy, output_shape = [10] )(x)
    k_var = K.placeholder(shape=(10,512), dtype='float32', name = 'lstm_1')
    tt = (x).eval(session=k.get_session())
    print(type(tt))
    y = cossy(tt)
    print("y: ",y)
#     return y #np.random.rand(10).astype('f').reshape(10,1)

In [418]:
def loss_func(y_true, y_pred):
    print("loss_func")
    y_true = cossimilasrity(y_true)
    y_pred = cossimilasrity(y_pred)
    print(y_true.shape)
    print(y_pred.shape)
    loss = objectives.mean_squared_error(y_true, y_pred)
    return loss

In [420]:
K.set_learning_phase(1)
model = Sequential()
model.add(TimeDistributed(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal", activation='relu'), input_shape=s1.shape))
#model.add(TimeDistributed())
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(64, (3,3),  padding='same', activation='relu')))
# model.add(TimeDistributed())
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Conv2D(32, (3,3),padding='same',activation='relu')))
# model.add(TimeDistributed())
model.add(BatchNormalization())
model.add(TimeDistributed(Conv2D(32, (3,3),padding='same',activation='relu')))
# model.add(TimeDistributed())
model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#freeze
# model.add(CustomRegularization())
#freeze
#model.add()
model.add(LSTM(6, return_sequences=False, dropout=0.2, name='lstm_2'))#E1 Objective Covered
model.add(Activation('relu'))
#model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#'categorical_crossentropy'
#compile E3 layerlstm
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1111 (TimeD (None, 10, 60, 60, 64)    4864      
_________________________________________________________________
batch_normalization_445 (Bat (None, 10, 60, 60, 64)    256       
_________________________________________________________________
time_distributed_1112 (TimeD (None, 10, 60, 60, 64)    36928     
_________________________________________________________________
batch_normalization_446 (Bat (None, 10, 60, 60, 64)    256       
_________________________________________________________________
time_distributed_1113 (TimeD (None, 10, 30, 30, 64)    0         
_________________________________________________________________
time_distributed_1114 (TimeD (None, 10, 30, 30, 64)    0         
_________________________________________________________________
time_distributed_1115 (TimeD (None, 10, 30, 30, 32)    18464     
__________

In [ ]:
model.fit(data, label, batch_size=1, epochs = 1)
# score, acc = model.evaluate(data, label)
# print('Test score:', score)
# print('Test accuracy:', acc)

In [28]:
#cnn mian seperate import
cnnmodel = Sequential()
cnnmodel.add(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(64, 64, 3)))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(Conv2D(64, 4))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))
cnnmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))
cnnmodel.add(Dropout(0.5))

cnnmodel.add(Conv2D(32, 3))
#cnnmodel.add(BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(Conv2D(32, 3))
cnnmodel.add(keras.layers.normalization.BatchNormalization())#BatchNormalization())
cnnmodel.add(Activation("relu"))

cnnmodel.add(MaxPool2D(pool_size=(2, 2), strides=2))
cnnmodel.add(Dropout(0.5))

cnnmodel.add(Flatten())
cnnmodel.add(Dense(512))
cnnmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnnmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 60, 60, 64)        4864      
_________________________________________________________________
activation_5 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 57, 57, 64)        65600     
_________________________________________________________________
activation_6 (Activation)    (None, 57, 57, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 26, 26, 32)        18464     
__________

In [14]:
## approach 2
model = Sequential()
sequence = Input(shape=[None, 64, 64, 3])
model.add(TimeDistributed(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal"), input_shape=(1000, 64, 64, 3)))
model.add(TimeDistributed(Conv2D(32, (3,3),  padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(BatchNormalization())
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=True, dropout=0.5))
model.add(Dense(6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 1000, 60, 60, 64)  4864      
_________________________________________________________________
time_distributed_9 (TimeDist (None, 1000, 58, 58, 32)  18464     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 1000, 29, 29, 32)  0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 1000, 29, 29, 64)  18496     
_________________________________________________________________
time_distributed_12 (TimeDis (None, 1000, 29, 29, 64)  36928     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 1000, 14, 14, 64)  0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 1000, 14, 14, 128) 73856     
__________